In [1]:
import os
import sys
import glob
import numpy as np
import pickle
import cv2
import open3d as o3d
import matplotlib.pyplot as plt
from projection_utils import Projection
from data_parser import read_keypoints
import time

sys.path.insert(1, '../')
from virtual_huams_resource.utils import get_smplx_body_model

sys.path.insert(0, "../virtual_huams_resource")
from virtual_huams_resource.benji_prox_dataloader import DatasetBase, proxDatasetSkeleton

In [3]:
data_root = "/cluster/scratch/bdayan/prox_data"
sequence_id = 'vicon_03301_01'
save_root = '../quantitative/3d_visualization'

smplx_model_path='../models_smplx_v1_1/models/'

In [4]:
vis = o3d.visualization.VisualizerWithKeyCallback()

In [5]:
# ______________ read skeletons ______________
pd2 = proxDatasetSkeleton(root_dir=data_root + '/PROXD', in_frames=45, pred_frames=0, output_type='raw_pkls',
                        search_prefix='results', extra_prefix='joints_worldnorm.pkl', frame_jump=10, window_overlap_factor=8)
idx, (a, in_skels), (b, fut_skels) = pd2.__getitem__(0)
sk = in_skels[0].detach().numpy()
sk.shape

(1, 25, 3)

In [8]:
# ______________ read point cloud ______________
scene_point_cloud = o3d.io.read_point_cloud(os.path.join(data_root, "scenes", "N3OpenArea.ply"))

In [9]:
LIMBS = [(23, 15),
         (24, 15),
         (15, 22),
         (22, 12),
         # left arm
         (12, 13),
         (13, 16),
         (16, 18),
         (18, 20),
         # right arm
         (12, 14),
         (14, 17),
         (17, 19),
         (19, 21),
         # spline
         (12, 9),
         (9, 6),
         (6, 3),
         (3, 0),
         # left leg
         (0, 1),
         (1, 4),
         (4, 7),
         (7, 10),
         # right leg
         (0, 2),
         (2, 5),
         (5, 8),
         (8, 11)]

In [ ]:
skeleton_iter = iter(in_skels)
def update(vis):
    skeleton = next(skeleton_iter)          # render first frame
    skeleton = skeleton.detach().numpy()
    skeleton = skeleton.astype(np.float64)
    skeleton = skeleton.squeeze()
    # print(skeleton)
    skeleton_input = o3d.geometry.LineSet(
            o3d.utility.Vector3dVector(skeleton),
            o3d.utility.Vector2iVector(LIMBS))
    ctrl = vis.get_view_control()
    cam_param = ctrl.convert_to_pinhole_camera_parameters()
    vis.clear_geometries()
    vis.add_geometry(scene_point_cloud)
    vis.add_geometry(skeleton_input)
    ctrl.convert_from_pinhole_camera_parameters(cam_param)
    vis.poll_events()
    vis.update_renderer()
    vis.run()
    
# ______________ animate skeleton in point cloud ______________
vis.create_window()
vis.register_key_callback(65, update)       # press A
vis.add_geometry(scene_point_cloud)

skeleton = in_skels[0].detach().numpy()          # render first frame
skeleton = skeleton.astype(np.float64)
skeleton = skeleton.squeeze()
skeleton_input = o3d.geometry.LineSet(
            o3d.utility.Vector3dVector(skeleton),
            o3d.utility.Vector2iVector(LIMBS))
vis.add_geometry(skeleton_input)
vis.run()
vis.destroy_window()